In [1]:
import os
import requests
import pandas as pd
import yfinance as yf
from dotenv import load_dotenv
import numpy as np
import alpaca_trade_api as tradeapi
from MCForecastTools import MCSimulation
%matplotlib inline

In [2]:
load_dotenv()

alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")


api = tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    api_version = "v2"
)

In [10]:
tickers = ["AGG", "VTI" , "GLD" ]


timeframe = "1D"


start_date = pd.Timestamp("2017-01-01", tz="America/New_York").isoformat()
end_date = pd.Timestamp("2021-12-31", tz="America/New_York").isoformat()


ticker_data = api.get_barset(
    tickers,
    timeframe,
    start=start_date,
    end=end_date,
    limit=1000,
).df

ticker_data = ticker_data.copy()
ticker_data.index.tz_convert(None)

ticker_data.tail()

AGG                                        GLD  \
                             open    high     low   close     volume    open   
time                                                                           
2021-12-27 00:00:00-05:00  114.15  114.30  114.13  114.27  5069840.0  168.98   
2021-12-28 00:00:00-05:00  114.43  114.45  114.17  114.22  4892072.0  169.35   
2021-12-29 00:00:00-05:00  113.99  114.05  113.81  113.87  5712959.0  167.36   
2021-12-30 00:00:00-05:00  113.99  114.13  113.83  114.12  5044053.0  168.43   
2021-12-31 00:00:00-05:00  114.15  114.31  114.06  114.09  6557778.0  170.53   

                                                                  VTI  \
                             high     low    close     volume    open   
time                                                                    
2021-12-27 00:00:00-05:00  169.42  168.78  169.370  4143614.0  240.35   
2021-12-28 00:00:00-05:00  169.65  168.62  168.655  4084750.0  243.06   
2021-12-29 00:00:00-05:00  168.69  167.28  168.590  5456250.0  242.55   
2021-12-30 00:00:00-05:00  169.81  168.37  169.790  4775006.0  243.11   
2021-12-31 00:00:00-05:00  171.04  170.04  170.960  6147669.0  242.09   

                                                                  
                               high      low    close     volume  
time                                                              
2021-12-27 00:00:00-05:00  242.9100  240.340  242.900  4167600.0  
2021-12-28 00:00:00-05:00  243.7800  242.042  242.365  3768903.0  
2021-12-29 00:00:00-05:00  243.1600  241.710  242.600  2786453.0  
2021-12-30 00:00:00-05:00  243.6900  242.000  242.200  4360256.0  
2021-12-31 00:00:00-05:00  242.6926  241.460  241.480  3259979.0

In [4]:

btc = yf.download('BTC-USD', '2017-01-01', '2021-12-31')
day = np.arange(1, len(btc) + 1)
btc['day'] = day
btc.drop(columns=['Adj Close', 'Volume'], inplace = True)
btc = btc[[ 'Open', 'High', 'Low', 'Close']]
btc.tail()

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close
Date,,,,
2021-12-27,50802.609375,51956.328125,50499.468750,50640.417969
2021-12-28,50679.859375,50679.859375,47414.210938,47588.855469
2021-12-29,47623.871094,48119.742188,46201.496094,46444.710938
2021-12-30,46490.605469,47879.964844,46060.312500,47178.125000
2021-12-31,47169.371094,48472.527344,45819.953125,46306.445312


In [5]:
columns=[('BTC','Open'),('BTC','High'),('BTC','Low'),('BTC','Close')]
btc.columns=pd.MultiIndex.from_tuples(columns)
btc.head()

BTC                                       
                   Open         High          Low        Close
Date                                                          
2017-01-01   963.658020  1003.080017   958.698975   998.325012
2017-01-02   998.617004  1031.390015   996.702026  1021.750000
2017-01-03  1021.599976  1044.079956  1021.599976  1043.839966
2017-01-04  1044.400024  1159.420044  1044.400024  1154.729980
2017-01-05  1156.729980  1191.099976   910.416992  1013.380005

In [6]:
pd.concat([ticker_data, btc], axis=1)
ticker_data.head()

TypeError: Cannot join tz-naive with tz-aware DatetimeIndex

In [ ]:
MC_dist = MCSimulation(portfolio_data = ticker_data, weights = [.80,.15,.05], num_simulation = 100, num_trading_days = 252*30)

In [ ]:
MC_dist.calc_cumulative_return()

In [ ]:
line_plot = MC_dist.plot_simulation()

In [ ]:
summarize = MC_dist.summarize_cumulative_return()

print(summarize)